# Quickstart

In this quickstart you will create a simple LLM Chain and learn how to log it and get feedback on an LLM response.

## Setup
### Add API keys
For this quickstart you will need Open AI and Huggingface keys

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

### Import from LangChain and TruLens

In [2]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import TruChain, Feedback, Huggingface, Tru
tru = Tru()

# imports from langchain to build app
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

### Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [3]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

### Send your first request

In [4]:
prompt_input = '¿que hora es?'

In [5]:
llm_response = chain(prompt_input)

display(llm_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: ¿que hora es?

> Finished chain.


{'prompt': '¿que hora es?',
 'text': '\n\nLa hora actual es [hora actual]. La hora es la medida en que la Tierra se divide en 24 horas del día. Una hora es un intervalo de tiempo que equivale a 60 minutos. Un minuto es una fracción de 60 segundos. El tiempo es medido en relojes, relojes digitales, teléfonos celulares y otros dispositivos.'}

## Initialize Feedback Function(s)

In [6]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on(
    text1="prompt", text2="response"
)

huggingface api: 0requests [00:00, ?requests/s]

## Instrument chain for logging with TruLens

In [7]:
truchain = TruChain(chain,
    chain_id='Chain3_ChatApplication',
    feedbacks=[f_lang_match],
    tru = tru)

✅ chain Chain3_ChatApplication -> default.sqlite
✅ feedback def. feedback_hash_26c84c76d907d808c36028e024af63c0 -> default.sqlite


In [8]:
# Instrumented chain can operate like the original:
llm_response = truchain(prompt_input)

display(llm_response)



> Entering new LLMChain chain...

> Finished chain.


{'prompt': '¿que hora es?',
 'text': '\n\nLa hora actual es la hora exacta del reloj. Dependiendo de dónde te encuentres, la hora puede ser diferente porque hay diferentes husos horarios en diferentes partes del mundo.'}

✅ record record_hash_c6e856f362dc9a30bc823182eeb335b4 from Chain3_ChatApplication -> default.sqlite


✅ feedback feedback_hash_26c84c76d907d808c36028e024af63c0 on record_hash_c6e856f362dc9a30bc823182eeb335b4 -> default.sqlite


## Explore in a Dashboard

In [9]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.run_dashboard(_dev=True) # if running from repo
# tru.stop_dashboard() # stop if needed

<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>


  You can now view your Streamlit app in your browser.

  Network URL: http://192.168.4.23:8502
  External URL: http://100.36.49.61:8502



### Chain Leaderboard

Understand how your LLM application is performing at a glance. Once you've set up logging and evaluation in your application, you can view key performance statistics including cost and average feedback value across all of your LLM apps using the chain leaderboard. As you iterate new versions of your LLM application, you can compare their performance across all of the different quality metrics you've set up.

Note: Average feedback values are returned and displayed in a range from 0 (worst) to 1 (best).

![Chain Leaderboard](Assets/image/Leaderboard.png)

To dive deeper on a particular chain, click "Select Chain".

### Understand chain performance with Evaluations
 
To learn more about the performance of a particular chain or LLM model, we can select it to view its evaluations at the record level. LLM quality is assessed through the use of feedback functions. Feedback functions are extensible methods for determining the quality of LLM responses and can be applied to any downstream LLM task. Out of the box we provide a number of feedback functions for assessing model agreement, sentiment, relevance and more.

The evaluations tab provides record-level metadata and feedback on the quality of your LLM application.

![Evaluations](Assets/image/Leaderboard.png)

### Deep dive into full chain metadata

Click on a record to dive deep into all of the details of your chain stack and underlying LLM, captured by tru_chain.

![Explore a Chain](Assets/image/Chain_Explore.png)

If you prefer the raw format, you can quickly get it using the "Display full chain json" or "Display full record json" buttons at the bottom of the page.

Note: Feedback functions evaluated in the deferred manner can be seen in the "Progress" page of the TruLens dashboard.

## Or view results directly in your notebook

In [10]:
tru.get_records_and_feedback(chain_ids=[])[0] # pass an empty list of chain_ids to get all

,record_id,chain_id,input,output,record_json,tags,ts,total_tokens,total_cost,chain_json,feedback_id,language_match
0,record_hash_c6e856f362dc9a30bc823182eeb335b4,Chain3_ChatApplication,¿que hora es?,\n\nLa hora actual es la hora exacta del reloj...,"{""chain"": {""_call"": {""args"": {""inputs"": {""prom...",dev,2023-05-30 13:31:25.534667,84,0.00168,"{""memory"": null, ""callbacks"": null, ""callback_...",feedback_hash_26c84c76d907d808c36028e024af63c0,0.993324
